## Purpose of script:
#### Reviewing Spark basics
#### Referencing Jose Portilla's "Spark and Python for Big Data with PySpark" course

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [3]:
df = spark.read.json('../Datasets/people.json')

df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [4]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [5]:
df.columns

['age', 'name']

In [6]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [7]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



#### Casting data types in Spark

In [9]:
from pyspark.sql.types import (StructField, StructType, 
                               StringType, IntegerType)

In [10]:
## True is for allowing for null values
data_schema = [StructField('age', IntegerType(), True),
               StructField('name', StringType(), True)]

In [11]:
specified_struc = StructType(fields=data_schema)

In [12]:
df = spark.read.json('../Datasets/people.json', schema=specified_struc)

In [13]:
## column types casted correctly
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)

